In [29]:
import numpy as np
import pandas as pd
import pickle
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.datasets import fetch_20newsgroups_vectorized
from sklearn.feature_selection import chi2, f_classif
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report, accuracy_score,log_loss
from sklearn import metrics

build function to find optimal n_estimators
note: if learning_rate, reg_alpha, reg_lambda are not set
xgboost will use default values from https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
e.g. eta = 0.3 instead of 0.1
if they are set, eta, alpha, lambda will be overwritten
the best n_estimators is Name + 1

In [2]:
# load data
with open("data_processed_test.pkl", "rb") as f:
    train_data = pickle.load(f)
    train_label = pickle.load(f)
    test_data = pickle.load(f)
    test_label = pickle.load(f)
    label_list = pickle.load(f)

In [3]:
label_list

['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer']

# Build model

# Multinomial regression

In [36]:
#fit a logistic regression model to the data
model = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, 
                           intercept_scaling=1, class_weight=None, random_state=None, 
                           solver='newton-cg', max_iter=100, multi_class='multinomial', 
                           verbose=0, warm_start=False, n_jobs=1)
model.fit(train_data, train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

# Attributes of model

In [37]:
feat_labels = train_data.columns[0:]
#feat_labels

In [38]:
#for f in range(train_data.shape[1]):
#    print("{0}. Feature {1}: {2}".format(f + 1, feat_labels[indices[f]], importances[indices[f]]))

In [39]:
print(model.coef_)

[[ -8.03298878e-01  -3.46753549e-02  -9.17035243e-02   9.17054533e-02
    3.70916592e-01  -2.23449036e-01   1.77116963e-02  -1.65177324e-01
   -1.89672857e-01  -2.06801565e-01  -7.19435673e-02  -1.56129503e-01
   -1.69464506e-01   4.80546355e-01   3.13467573e-01   1.48727812e+00
   -1.48727619e+00   1.09322896e-01  -1.09320967e-01   1.16366714e-02
    1.56926371e-01  -1.68561114e-01   1.55765105e-01   2.45564980e-01
    6.85112516e-03  -5.73840610e-01  -9.55376099e-03   3.78069028e-01
    1.66511182e-02   2.41740061e-03  -2.21922457e-01  -3.38619077e-02
    3.38638366e-02]
 [  1.50307159e-01   1.34292238e-01   7.28725546e-01  -7.28733135e-01
   -5.55148268e-01  -3.03207838e-01   4.97183620e-01   3.61164898e-01
    3.11822025e-01   1.69780655e-01  -4.93214261e-01  -2.58179951e-01
    5.89185591e-02   3.48872603e-02   1.75978125e-01  -6.13854227e-01
    6.13846639e-01   9.90890146e-03  -9.91648959e-03  -3.48525550e-02
    6.27220928e-04   3.42177460e-02  -1.97832836e-01  -1.71578950e-01


In [40]:
print(model.intercept_)

[ 1.21762864 -2.86394576 -0.74539423 -0.08372428  2.47543563]


In [41]:
scores, pvalues1 = chi2(train_data, train_label)

In [42]:
pvalues1

array([  1.41272216e-211,   7.02078700e-004,   5.48541483e-197,
         1.73474633e-038,   5.24283374e-095,   1.83398114e-074,
         1.48110566e-085,   1.95815345e-056,   5.13527489e-002,
         2.52321689e-005,   4.82706905e-006,   2.63116022e-007,
         3.87966818e-007,   4.51608422e-023,   6.56921407e-023,
         8.46469619e-110,   0.00000000e+000,   2.39102255e-005,
         1.13854630e-004,   3.91469451e-002,   5.95597274e-022,
         1.11948270e-015,   1.74573125e-006,   5.39135076e-006,
         4.32949522e-004,   4.86635975e-059,   7.54828120e-003,
         4.72812302e-004,   1.09438672e-007,   2.30737878e-001,
         2.08370000e-002,   1.85641603e-002,   1.63047178e-001])

In [43]:
F, pvalues2 = f_classif(train_data, train_label)

In [44]:
pvalues2

array([  4.83411753e-314,   8.95306825e-007,   8.82109338e-248,
         8.82109338e-248,   2.69704370e-192,   1.69563841e-144,
         8.68510328e-090,   3.85491159e-058,   2.68235082e-002,
         3.49405604e-006,   5.96550547e-007,   1.90189277e-008,
         2.80350209e-008,   9.02028824e-028,   8.87826562e-028,
         0.00000000e+000,   0.00000000e+000,   3.75518082e-010,
         3.75518082e-010,   1.45231094e-007,   1.36194036e-027,
         5.21268837e-017,   7.95613508e-008,   1.53077415e-006,
         2.40480854e-004,   9.04438511e-070,   2.52665336e-003,
         2.03823874e-004,   7.38292671e-010,   2.27090905e-001,
         1.23661978e-002,   1.04102577e-003,   1.04102577e-003])

In [16]:
# fit a logistic regression model to the data
model1 = LogisticRegression(penalty='l2', dual=False, tol=0.000001, C=1.0, fit_intercept=True, 
                           intercept_scaling=1, class_weight=None, random_state=None, 
                           solver='lbfgs', max_iter=10000, multi_class='multinomial', 
                           verbose=0, warm_start=False, n_jobs=1)
model1.fit(train_data, train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=1e-06, verbose=0, warm_start=False)

In [27]:
print(model1.coef_)

[[ -8.02093432e-01  -3.40448717e-02  -9.21101242e-02   9.10421763e-02
    3.70242271e-01  -2.24685565e-01   1.63499086e-02  -1.62974563e-01
   -1.91052738e-01  -2.07649602e-01  -7.10646953e-02  -1.56694407e-01
   -1.68920280e-01   4.80607343e-01   3.13706432e-01   1.48667587e+00
   -1.48774382e+00   1.08760450e-01  -1.09828398e-01   1.12097818e-02
    1.56742169e-01  -1.69019898e-01   1.53400655e-01   2.46046167e-01
    4.49816738e-03  -5.75389539e-01  -1.18448612e-02   3.79092842e-01
    1.57424942e-02   1.22938892e-02  -2.24907763e-01  -3.44584716e-02
    3.33905237e-02]
 [  1.45593060e-01   1.31558480e-01   7.24958992e-01  -7.30963959e-01
   -5.58241323e-01  -3.04415382e-01   4.95665117e-01   3.60986621e-01
    3.15908406e-01   1.71564958e-01  -4.98203294e-01  -2.57350638e-01
    5.53427884e-02   3.31393247e-02   1.73593487e-01  -6.16721311e-01
    6.10716344e-01   7.09724938e-03  -1.31022163e-02  -3.70710681e-02
   -1.08757516e-03   3.21536763e-02  -1.89499700e-01  -1.72221553e-01


In [25]:
print(model1.n_iter_)

[309]


In [45]:
model.coef_

array([[ -8.03298878e-01,  -3.46753549e-02,  -9.17035243e-02,
          9.17054533e-02,   3.70916592e-01,  -2.23449036e-01,
          1.77116963e-02,  -1.65177324e-01,  -1.89672857e-01,
         -2.06801565e-01,  -7.19435673e-02,  -1.56129503e-01,
         -1.69464506e-01,   4.80546355e-01,   3.13467573e-01,
          1.48727812e+00,  -1.48727619e+00,   1.09322896e-01,
         -1.09320967e-01,   1.16366714e-02,   1.56926371e-01,
         -1.68561114e-01,   1.55765105e-01,   2.45564980e-01,
          6.85112516e-03,  -5.73840610e-01,  -9.55376099e-03,
          3.78069028e-01,   1.66511182e-02,   2.41740061e-03,
         -2.21922457e-01,  -3.38619077e-02,   3.38638366e-02],
       [  1.50307159e-01,   1.34292238e-01,   7.28725546e-01,
         -7.28733135e-01,  -5.55148268e-01,  -3.03207838e-01,
          4.97183620e-01,   3.61164898e-01,   3.11822025e-01,
          1.69780655e-01,  -4.93214261e-01,  -2.58179951e-01,
          5.89185591e-02,   3.48872603e-02,   1.75978125e-01,
       

# Multinomial regression with SGD

In [20]:
model2 = SGDClassifier(loss='log', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, 
                       n_iter=100, shuffle=True, verbose=0, epsilon=0.1, n_jobs=1, random_state=None, 
                       learning_rate='optimal', eta0=0.0, power_t=0.5, class_weight=None, warm_start=True, 
                       average=False)

In [22]:
model2.fit(train_data, train_label)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=100, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=True)

In [26]:
print(model2.coef_)

[[-0.8125398   0.0890214   0.26862095  0.10295438  0.57245604 -0.08286845
   0.78356641 -0.90157868 -0.02777989 -0.2488617  -0.20784676 -0.19378266
  -0.09054455  0.60453119  0.5358597   1.94900011 -1.57742478  0.30002034
   0.07155499  0.10010341  0.47180494 -0.20033301  0.18846821  0.18756667
  -0.04159419 -0.38432038  0.01201274  0.23818487  0.1872765   0.09283037
  -0.10884946  0.12847604  0.24309929]
 [ 0.32349698  0.19815175  0.95915015 -0.60020549 -0.31668002 -0.11718771
   0.73329348  0.05951891  0.40458991  0.26659648 -0.47866152 -0.26646509
   0.104043    0.09256029  0.23628158 -0.54444365  0.90338831  0.16892427
   0.19002039  0.13790755  0.11505027  0.10598684 -0.22229362 -0.21647498
  -0.23697988  0.47868899  0.22728034 -0.65985082  0.90891475 -0.11478478
   0.19444465  0.17134789  0.18759677]
 [ 1.04632776  0.1192822   0.80014151 -0.19667047  0.06607995  0.09953855
   2.0793147  -1.64146216  0.26870574  0.12755041  0.27365833  0.31056503
   0.20503013 -0.3903282  -0.19171

# Make prediction

In [9]:
predict_label = model.predict(test_data)

In [10]:
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

NameError: name 'expected' is not defined

In [ ]:
predict_label

In [ ]:
accuracy = cross_val_score(model, train_data, train_label, cv=7)
print("Accuracy: %0.2f (+/- %0.2f)" % (accuracy.mean(), accuracy.std() * 2))

In [ ]:
#model.score(test_data, test_label)

In [ ]:
metrics.precision_recall_fscore_support(test_label, predict_label, beta=0.5, average=None)

In [12]:
label_probs = model.predict_proba(test_data)
predict_label = model.predict(test_data)
#y_probs = model.predict_log_proba(test_data)

In [13]:
print("accuracy_score:", accuracy_score(test_label, predict_label))
print("log_loss:", log_loss(test_label, label_probs))

accuracy_score: 0.574516129032
log_loss: 1.01359493281


In [15]:
model.fit(train_data, train_label)
importance = model.feature_importances_
sns.barplot(y=dog_train.columns, x=importance)

AttributeError: 'LogisticRegression' object has no attribute 'feature_importances_'

In [ ]:
log_probability = pd.DataFrame(y_probs,columns =  ["Adoption","Died","Euthanasia","Return_to_owner","Transfer"])

In [ ]:
log_probability

In [ ]:
print(classification_report(test_label,predict_label))
print(accuracy_score(test_label,predict_label))
print(log_loss(test_label,y_probs))